# Adversarial thresholding semi-bandits
## Empirical analysis

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
import os

## Import ALICE TRD Anode current dataset

In [ ]:
os.chdir('/home/bower5005/ALICE_TRD/ALICE_TRD_HV')
Yc_ = np.load(os.getcwd()+'/Yc_.npz')
Yc_ = Yc_['arr_0']
T = np.shape(Yc_)[0]
K = np.shape(Yc_)[1]

## Plot dataset

In [ ]:
plt.figure(figsize=(30,10))
font_size = 25

plt.plot(Yc_, color='blue', alpha=0.2)
plt.plot(Yc_[:,3], color='red')

ITRIPLimit = np.zeros(T)
ITRIPLimit[:] = np.nan
ITRIPLimit[:17533] = 12/np.float(15)
ITRIPLimit[17533:] = 20/np.float(15)
plt.plot(ITRIPLimit, color='black')

plt.xlim([0,T])
plt.ylim([0,20.1/np.float(15)])

plt.xlabel('Time stamp', fontsize=font_size)
plt.ylabel(r'TRD Anode Current, $\mu\,$A', fontsize=font_size)
plt.tick_params(labelsize=font_size)
plt.rc('font', size=font_size)
#plt.ticklabel_format(style='sci', axis='x', scilimits=(0,0), useMathText=True)
dates = ['2018-06-09 12:00:00', '18:00:00', '2018-06-10 00:00:00', '06:00:00', '12:00:00', '18:00:00', '2018-06-11 00:00:00']
dates = ['12:00:00 \n09-06-2018', '18:00:00', '00:00:00 \n10-06-2018', '06:00:00', '12:00:00', '18:00:00', '00:00:00 \n11-06-2018']
plt.xticks(np.arange(0,T+1,T/np.float(6)), dates, rotation=45, fontsize=font_size, horizontalalignment='right')
plt.xticks(np.arange(0,T+1,T/np.float(6)), dates, fontsize=font_size, rotation=0, horizontalalignment='center')
plt.yticks(np.arange(0,1.1+1/np.float(3),1/np.float(3)), [0,5,10,15,20], fontsize=font_size)
plt.show()

## Import empirical results

In [ ]:
os.chdir('/home/bower5005/Downloads')
files = sorted(os.listdir())
for i in range(len(files)):
    print(i, files[i])

## Plot ATSBP regret

In [ ]:
length = np.int(0.5*len(files))
REGRET = [[] for i in range(length)]
for i in range(length):
    REGRET[i] = np.load(os.getcwd()+'/'+str(files[i+length]))
    REGRET[i] = REGRET[i]['arr_0']

if REGRET[0].shape[1] == 50000:
    T = 50000
else:
    T=64800

if REGRET[0].shape[0] == 10 and REGRET[0].shape[1] == T:
    for i in range(length):
        REGRET[i] = np.transpose(REGRET[i])

Fixed_T1 = np.zeros([T,length])
Fixed_T2 = np.zeros([T,length])
confidence_gap = 5000
position = np.arange(0,T,confidence_gap)
for i in range(length):
    Fixed_T1[:,i] = np.median(REGRET[i], axis=1)
    q1 = np.percentile(REGRET[i], 75, axis=1)
    q2 = np.percentile(REGRET[i], 25, axis=1)
    e_bar = np.array(q1 - q2)
    Fixed_T2[:,i] = np.nan
    Fixed_T2[np.arange(0,T,confidence_gap),i] = e_bar[np.arange(0,T,confidence_gap)]

In [ ]:
plt.figure(figsize=(10,10))
Rounds = np.arange(T)
font_size = 25
colors_ = ['blue', 'red', 'yellowgreen', 'magenta', 'gold', 'black', 'purple', 'orange']

#Labels = [r'LET-Exp3.M, $\epsilon=0.7$', r'LET-Exp3.M, $\epsilon=0.8$', r'LET-Exp3.M, $\epsilon=0.9$', r'T-Exp3.M, $\epsilon=1$']
#Labels = ['k=5', 'k=10', 'k=15', 'k=20']
Labels = [r'$t_{max}=1\,$min', r'$t_{max}=5\,$mins', r'$t_{max}=10\,$mins', r'$t_{max}=15\,$mins', r'$t_{max}=30\,$mins', r'$t_{max}=60\,$mins']
#Labels = [r'$t_{max}=1\,$min', r'$t_{max}=10\,$mins', r'$t_{max}=30\,$mins', r'$t_{max}=60\,$mins']
#Labels = [r'AliceBandit, $t_{max}=1\,$min', r'AliceBandit, $t_{max}=5\,$mins', r'AliceBandit, $t_{max}=10\,$mins', r'AliceBandit, $t_{max}=15\,$mins', r'AliceBandit, $t_{max}=30\,$mins', r'AliceBandit, $t_{max}=60\,$mins',r'T-Exp3.M, $\epsilon=1$',r'dLET-Exp3.M, $\epsilon=1$, k=15']

for i in range(length):
    plt.errorbar(Rounds, Fixed_T1[:,i], yerr=Fixed_T2[:,i], capsize=2, color=colors_[i], label=str(Labels[i]), linewidth=1)
    
x_max = T
y_max = np.ceil((np.max(Fixed_T1)+np.nanmax(Fixed_T2))/25)*25
y_max = 1500

plt.xlim([0,x_max])
plt.ylim([0,y_max])

#plt.yticks(np.arange(0,y_max+1,y_max*0.25), fontsize=font_size)
plt.tick_params(labelsize=font_size)
plt.rc('font', size=font_size)
plt.ticklabel_format(style='sci', axis='x', scilimits=(0,0), useMathText=True)
#plt.ylabel('Cumulative regret', fontsize=font_size)
plt.ylabel('Cumulative reward', fontsize=font_size)
plt.xlabel('Round', fontsize=font_size)

plt.legend(loc='best', fontsize=font_size)
plt.tight_layout()
plt.show()

In [ ]:
length = np.int(0.5*len(files))
CUMP = [[] for i in range(length)]
for i in range(length):
    CUMP[i] = np.load(os.getcwd()+'/'+str(files[i]))
    CUMP[i] = CUMP[i]['arr_0']

## Plot Cumulative pull frequency

In [ ]:
n_row = 2
n_col = 3
size = 10
fig, ax = plt.subplots(n_col,n_row, figsize=(n_row*size,n_col*size))
font_size = 25
plt.tick_params(labelsize=font_size)
plt.rc('font', size=font_size)
count = 0
colors_ = ['red', 'yellowgreen', 'magenta', 'orange', 'purple']
for i in range(n_col):
    for j in range(n_row):
        a = np.mean(CUMP[count], axis=0)
        if T == 64800:
            arms = [3,4,27]
        elif T == 50000:
            arms = [0,1,2,3,4]
        for arm in arms:
            ax[i,j].plot(a[:,arm], color=colors_[arm], alpha=1, label='Arm {}'.format(arm))
        ax[i,j].plot(a, color='blue', alpha=0.2)
        ax[i,j].plot(a[:,5], color='blue', alpha=0.2, label='Arms 5-9')
        #ax[i,j].axvline(x=14465, color='black', linewidth=1)
        #ax[i,j].axvline(x=17533, color='black', linewidth=1)
        #ax[i,j].axvline(x=28928, color='black', linewidth=1)
        
        x_max = T
        if i == 0 and j == 0:
            y_max = 2250
        else:
            y_max = 1500
        ax[i,j].set_xlim([0,x_max])
        ax[i,j].set_ylim([0,y_max])
        ax[i,j].set_xticks(np.arange(0,x_max+1,x_max*0.25))
        ax[i,j].set_yticks(np.arange(0,y_max+1,y_max*0.25))
        ax[i,j].ticklabel_format(style='sci', axis='x', scilimits=(0,0), useMathText=True)
        ax[i,j].set_ylabel('Cumulative pull frequency')
        ax[i,j].set_xlabel('Round', fontsize=font_size)
        count += 1
ax[0,0].legend(loc='best')
plt.tight_layout()